# Model Selection
We will be focusing on XGBoost, since it achieved the best score in the eda steps.
Possibly the best solution would involve a combination of some neural network, with XGBoost 
and/or LightGBM. More details on the steps and decisions can be found in the data_eda.ipynb notebook

## Imports 

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
import xgboost as xgb
import notebook_utils as nu 
import numpy as np
import joblib

random_state = 42

## Loads Data and Separates into Train and Test sets

In [2]:
data = pd.read_csv('./data/train.csv')

In [3]:
data.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [4]:
x_cols = [x for x in data.columns if (x not in ['ID_code', 'target'])]

X = data[x_cols]
y = data['target']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=random_state)

### Oversample train data

In [6]:
y_train.head()

38762     0
76883     0
2018      0
133899    0
170373    1
Name: target, dtype: int64

In [7]:
X_train.head()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
38762,12.6508,-1.4518,4.9398,9.4263,11.3132,-3.2654,5.3040,20.0105,-0.5506,7.4902,...,0.9493,10.1718,1.8558,12.2295,22.8003,0.1811,1.0184,8.9432,10.7889,-15.1560
76883,8.8434,-5.0525,5.8053,2.5540,9.7131,1.8681,5.3080,14.4369,4.9028,6.9503,...,5.2870,10.8789,2.6776,-5.2744,19.9168,-0.5264,-3.4889,9.4107,10.8898,6.6960
2018,5.8905,3.8340,7.7051,8.8097,12.6723,0.0678,4.1572,14.7936,-0.2814,9.8768,...,4.1241,4.4696,5.0724,1.5313,17.1286,0.9120,0.8431,8.8373,16.0513,-11.9798
133899,8.2786,5.8034,13.2013,8.9956,13.5608,-10.0445,5.1545,16.3067,-3.5246,10.0120,...,0.7748,14.6195,3.0610,-5.9456,15.4801,0.2584,-1.0892,7.4688,16.2685,-10.2297
170373,7.3882,6.3704,9.9413,7.9744,11.6259,-8.7820,4.4755,17.7317,3.7531,8.8027,...,9.1395,5.5115,3.3208,0.3378,19.2325,-2.2717,7.2787,9.0987,15.9157,-17.2046


In [8]:
df0_indexes = y_train.index[y_train == 0].tolist()
df1_indexes = y_train.index[y_train == 1].tolist()

X_train_0 = X_train.loc[df0_indexes]
y_train_0 = y_train.loc[df0_indexes]

nr_df_0 = X_train_0.shape[0]

X_train_1 = X_train.loc[df1_indexes]
y_train_1 = y_train.loc[df1_indexes]

n_samples = nr_df_0
X_train_1, y_train_1 = resample(X_train_1, y_train_1, replace=True, n_samples=n_samples,random_state=random_state)

X_train = pd.concat([X_train_0, X_train_1])
y_train = pd.concat([y_train_0, y_train_1])

In [9]:
X_train.describe()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
count,323870.000000,323870.000000,323870.000000,323870.000000,323870.000000,323870.000000,323870.000000,323870.000000,323870.000000,323870.000000,...,323870.000000,323870.000000,323870.000000,323870.000000,323870.000000,323870.000000,323870.000000,323870.000000,323870.000000,323870.000000
mean,10.892665,-1.365485,10.910729,6.825571,11.097798,-4.748479,5.488130,16.542181,0.365201,7.501028,...,3.575732,7.623189,1.842769,3.236723,17.898590,-0.093978,2.473966,8.864975,15.659997,-2.963305
std,3.150621,4.136830,2.740539,2.060147,1.638567,7.986068,0.896579,3.420468,3.333813,1.256785,...,4.688064,3.113859,1.484306,4.024613,3.165741,1.455688,5.496634,0.935680,3.123425,10.446711
min,0.452800,-15.043400,2.117100,-0.040200,5.074800,-32.562600,2.347300,5.349700,-10.505500,3.970500,...,-14.093300,-2.691700,-3.814500,-11.783400,8.694400,-5.261000,-14.209600,5.960600,6.299300,-38.852800
25%,8.559500,-4.533075,8.826400,5.271950,9.889500,-10.944625,4.819200,13.921600,-2.227200,6.551700,...,0.167200,5.270500,0.788700,0.497800,15.529000,-1.150975,-1.765575,8.213500,13.573900,-10.790950
50%,10.726800,-1.338600,10.792000,6.864550,11.126000,-4.513600,5.477700,16.463800,0.479650,7.571600,...,3.557100,7.543400,1.790900,3.301000,17.858800,-0.135000,2.600900,8.846400,15.719500,-2.466400
75%,13.014100,1.646200,12.731000,8.352900,12.291100,1.223200,6.100500,19.082500,3.012400,8.524375,...,6.843400,9.718300,2.868500,6.140900,20.318400,0.892900,6.728000,9.562900,17.942875,5.249200
max,20.315000,10.376800,19.353000,13.188300,16.671400,17.251600,8.355600,27.691800,10.151300,11.150600,...,18.440900,16.684600,8.402400,18.281800,27.928800,4.272900,18.321500,12.000400,26.079100,28.500700


In [10]:
y_train.describe()

count    323870.000000
mean          0.500000
std           0.500001
min           0.000000
25%           0.000000
50%           0.500000
75%           1.000000
max           1.000000
Name: target, dtype: float64

## 1) Train Single XGBoost Instance

### 1.1) Hyperparameters:

In [11]:
hyperparameters = {
    'learning_rate': 0.3, # default
    'max_depth': 6, # default
    'min_child_weight': 1, # default
#     'subsample': 0.75,
#     'colsample_bytree': 0.75,
    'random_state': random_state
}

### 1.3) Train:

In [12]:
xgb_model = xgb.XGBClassifier(learning_rate=hyperparameters['learning_rate'],
                              n_jobs=-1,
                              max_depth=hyperparameters['max_depth'],
                              min_child_weight=hyperparameters['min_child_weight'],
#                               subsample=hyperparameters['subsample'],
#                               colsample_bytree=hyperparameters['colsample_bytree'],
                              objective='binary:logistic',
                              random_state=random_state,
                              verbosity=1
                              )
xgb_model.fit(X_train, y_train)

/home/ffreis/anaconda3/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[02:19:02] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.3, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=-1, num_parallel_tree=1,
              objective='binary:logistic', random_state=42, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', use_label_encoder=True,
              validate_parameters=1, verbosity=1)

In [13]:
print(xgb_model.get_xgb_params())
xgb.XGBClassifier(params = xgb_model.get_xgb_params())

{'objective': 'binary:logistic', 'base_score': 0.5, 'booster': 'gbtree', 'colsample_bylevel': 1, 'colsample_bynode': 1, 'colsample_bytree': 1, 'gamma': 0, 'gpu_id': -1, 'interaction_constraints': '', 'learning_rate': 0.3, 'max_delta_step': 0, 'max_depth': 6, 'min_child_weight': 1, 'monotone_constraints': '()', 'n_jobs': -1, 'num_parallel_tree': 1, 'random_state': 42, 'reg_alpha': 0, 'reg_lambda': 1, 'scale_pos_weight': 1, 'subsample': 1, 'tree_method': 'exact', 'validate_parameters': 1, 'verbosity': 1}


XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, gamma=None,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None...
                      'n_jobs': -1, 'num_parallel_tree': 1,
                      'objective': 'binary:logistic', 'random_state': 42,
                      'reg_alpha': 0, 'reg_lambda': 1, 'scale_pos_weight': 1,
                      'subsample': 1, 'tree_method': 'exact',
                      'validate_parameters': 1, 'verbosity': 1},
              random_state=None, reg_alpha=None, reg_lambda=None,
              scale_pos_weight=None, subsample=None, tree_method=None,
              use_label_encoder=True, validate_parameters=None, verbosity=None)

### 1.4) Evaluate the Results:

In [18]:
y_pred = xgb_model.predict(X_test)
y_true = np.array(y_test)

scores = nu.get_scores(y_true, y_pred)
nu.print_scores(scores)

accuracy score: 0.86595
gini score: 0.5079081009615076
f1 score: 0.48193236714975846
precision score: 0.3968809675366009
recall score: 0.6133792424987703


### 1.4) Save Model

In [22]:
joblib.dump(xgb_model, "xgb_model.joblib.dat")

['xgb_model.joblib.dat']

### 1.5) Tests if model was correctly saved

In [18]:
xgb_model = pickle.load(open("xgb_model.pickle.dat", "rb"))

In [19]:
y_pred = xgb_model.predict(X_test)
y_true = np.array(y_test)

scores = nu.get_scores(y_true, y_pred)
nu.print_scores(scores)

accuracy score: 0.86595
gini score: 0.5079081009615076
f1 score: 0.48193236714975846
precision score: 0.3968809675366009
recall score: 0.6133792424987703


## 2) First Hyperparameter Selection/Tuning
TODO: make GridSearchCV

In [13]:
'learning_rate': [0.01, 0.03, 0.1],
        'max_depth': [3, 5, 7, 10, 15],
        'min_child_weight': [1, 3, 5, 10],
        'subsample': [0.3, 0.5, 0.75, 1],
        'colsample_bytree': [0.3, 0.5, 0.75, 1],
        'n_estimators' : [50, 100, 200, 500, 1000],
        'objective': ['reg:squarederror']

SyntaxError: invalid syntax (<ipython-input-13-326acb4f6f3b>, line 1)

## 3) Second Hyperparameter Selection
TODO
The second iteration is made based on the first